# Get PheWAS input files

To perform EBV DNAemia PheWAS within the AoU cohort, we followed the demo workspace for performing PheWAS in AoU: https://workbench.researchallofus.org/workspaces/aou-rw-269aaeea/demophewassmoking/data. 

This script queries for the demographics and EHR data (ICD9/10 codes), along with the ICD code annotation files, that are used in downstream analyses.

In [ ]:
# Python packages
import os
import pandas as pd

In [ ]:
my_bucket = os.getenv('WORKSPACE_BUCKET')
my_bucket

In [ ]:
CDR_version=os.getenv("WORKSPACE_CDR")

In [ ]:
# Get ICD code annotation files
os.system(f'gsutil -m cp {my_bucket}/notebooks/Phecode_map_v1_2_icd10cm_beta.csv '+ 'Phecode_map_v1_2_icd10cm_beta.csv')
os.system(f'gsutil -m cp {my_bucket}/notebooks/ICDPhecodes ' + 'ICDPhecodes')

# These files did not end up being used in the downstream analysis
# os.system(f'gsutil -m cp {my_bucket}/notebooks/ICD9PhecodeExclude.csv ' + 'ICD9PhecodeExclude.csv')
# os.system(f'gsutil -m cp {my_bucket}/notebooks/phecode_map_icd9_10.csv ' + 'phecode_map_icd9_10.csv')
# os.system(f'gsutil -m cp {my_bucket}/notebooks/pheinfo.csv ' + 'pheinfo.csv')
# os.system(f'gsutil -m cp {my_bucket}/notebooks/phecode_rollup_map.csv ' + 'phecode_rollup_map.csv')

In [ ]:
# Extract relevant information from condition_occurrence table 
## Grouped by person and ICD code
query="""SELECT person_id,
                condition_source_value,
                COUNT(*) as n
         FROM `"""+CDR_version+""".condition_occurrence`
         GROUP BY person_id, condition_source_value
         ORDER BY person_id"""
condition=pd.read_gbq(query, dialect="standard")
# Save output as csv
condition.to_csv("condition_source_df.csv", sep=',')

In [ ]:
# Extract relevant information from observation table 
## Filtered to ICD9/ICD10 codes by mapping to concept table
## Grouped by person and ICD code
query="""SELECT person_id,
                observation_concept_id,
                observation_source_concept_id,
                observation_source_value,
                COUNT(*) as n
         FROM `"""+CDR_version+""".observation` as obs
         LEFT JOIN `"""+CDR_version+""".concept` as concept
             ON obs.observation_source_concept_id=concept.concept_id 
         WHERE vocabulary_id IN ('ICD9CM','ICD10CM')
         GROUP BY person_id,
                observation_concept_id,
                observation_source_concept_id,
                observation_source_value"""
obs=pd.read_gbq(query, dialect="standard")
# Save output as csv
obs.to_csv("observation_df.csv", sep=',')

In [ ]:
# Extract entire demographic table
query="""SELECT *
         FROM `"""+CDR_version+""".person`"""
demo=pd.read_gbq(query, dialect="standard")
# Save output as csv
demo.to_csv("demographic_df.csv", sep=',')